## The CUDA Execution Model

CUDA programming model exposes two primary abstractions:
- A memory hierarchy
- A thread hierarchy

These abstractions allow us to control the massively parallel GPU. 


### GPU Architecture Overview

- The GPU architecture is built around a scalable array of **Streaming Multiprocessors (SM)**

- Each SM in a GPU is designed to support concurrent execution of hundreds of threads, and there are generally multiple SMs per GPU
  - So it is possible to have 1000s of threads executing concurrently on a single GPU.
  
- CUDA employs a **Single Instruction Multiple Thread (SIMT)** architecture to manage and execute threads. 

   - There can be as many as 32 CUDA threads (aka **warp**) in-flight running on same CUDA core.
   
- SIMT vs SIMD (Single Instruction Multiple Data)
   - Both implement parallelism by broadcasting the same instruction to muliple execution units
   - SIMD requires that all vector elements in a vector execute together in a unified synchronous group, whereas SIMT allows multiple threads in the same warp to execute indipendently. 
   - SIMT model includes three key features that SIMD does not
     - Each thread has its own instruction address counter
     - Each thread has its own register state
     - Each thread can have an independent execution path

![](../images/fermi-sm-architecture.png)

The key components of a Fermi SM: 

- ➤ CUDA Cores 
- ➤ Shared Memory/L1 Cache 
- ➤ Register File 
- ➤ Load/Store Units 
- ➤ Special Function Units 
- ➤ Warp Scheduler

- A thread block is scheduled on only one SM.
- Once a thread block is scheduled on an SM, it remains there until execution completes
- An SM can hold more than one thread block at the same time

![](../images/logical-view-and-haardware-view.png)

- Shared memory and registers are precious resources in an SM
- Shared memory is partitioned among **thread blocks resident on the SM** and registers are partititioned among **threads**.
- Threads in a thread block can cooperate and communicate with each other through these resources.
- sharing data among parallel threads may cause a race condition
  - CUDA provides a means to synchronize threads within a thread block to ensure all threads reach certain points in execution before making further progress.
  - However, no primitives are provided for inter-block synchronization
    

## Understanding the Nature of Warp Execution

### Wraps and Thread Blocks

- Warps are the basic unit of execution in an SM
- When you launch a grid of thread blocks, the thread blocks in the grid are distributed among SMs.
- Once a thread block is scheduled to an SM, threads in the thread block are further partitioned into warps
- A warp consists of 32 consecutive threads
   - All threads in a warp are executed in SMIPT fashion
   - All threads execute the same instruction, and each thread carries out that operation on its own private data

![](../images/warp.png)

- Thread blocks can be configured to be **1D, 2D, or 3D**
- However, from the hardware perspective, all threads are arranged one-dimensionally
- Each thread has a unique ID in a block
- For a 1-D thread block, the unique thread ID is stored in the CUDA built-in variable `threadIdx.x`
- Threads with consecutive values for `threadIdx.x` are grouped into warps
- For example, 1-D thread block with 128 threads will be organized into 4 warps:
    
```
Warp 0: thread 0, thread 1, thread 2, ... thread 31 
Warp 1: thread 32, thread 33, thread 34, ... thread 63 
Warp 2: thread 64, thread 65, thread 66, ... thread 95 
Warp 3: thread 96, thread 97, thread 98, ... thread 127
```

- The logical layout of a 2-D or 3-D thread block can be converted into its one-dimensional physical layout by using the `x` dimension as the innermost dimension, the `y` dimension as the second dimension, and the `z` dimension as the outermost.

- For example, given a 2-D thread block, a unique identifier for each thread in a block can be calculated using the built-in `threadIdx` and `blockDim` variables:

    `threadIdx.y * blockDim.x  + threadIdx,x`
    
- The same calculation for a 3-D thread block is as follows:

    `threadIdx.z * blockDim.y * blockDim.x + threadIdx.y * blockDim.x + threadIdx.x`
    
- The number of warps for a thread block can be determined as follows:

$$\text{WarpsPerBlock}=ceil(\frac{\text{ThreadsPerBlock}}{\text{warpSize}})$$

### Warp Divergence

- Wrap divergence occurs when threads within a wrap take different code paths.
- Different `if-then-else` branches are executed serially.
- Try to adjust the granularity to be a multiple of warp size to avoid warp divergence.
- Different warps can execute different code with no penalty on performance



### Latency Hiding

**THROUGHPUT AND BANDWIDTH**

- Bandwidth and throughput are often confused, but may be used interchangeably depending on the situation. Both throughput and bandwidth are rate metrics used to measure performance.
- Bandwidth is usually used to refer to a theoretical peak value, while throughput is used to refer to an achieved value.
- Bandwidth is usually used to describe the highest possible amount of data transfer per time unit, while throughput can be used to describe the rate of any kind of information or operations carried out per time unit, such as, how many instruc- tions are completed per cycle.


**EXPOSING SUFFICIENT PARALLELISM**

- Because the GPU partitions compute resources among threads, switching between concurrent warps has very little overhead (on the order of one or two cycles) as the required state is already available on-chip. If there are sufficient concurrently active threads, you can keep the GPU busy in every pipeline stage on every cycle. In this situation, the latency of one warp is hidden by the execution of other warps. Therefore, exposing sufficient parallelism to SMs is beneficial to performance.
- A simple formula for calculating the required parallelism is to multiply the number of cores per SM by the latency of one arithmetic instruction on that SM. For example, Fermi has 32 single-precision floating-point pipeline lanes and the latency of one arithmetic instruction is 20 cycles, so at minimum 32 x 20 = 640 threads per SM are required to keep your device busy. However, this is a lower bound.


### Occupancy

**GUIDELINES FOR GRID AND BLOCK SIZE**
Using these guidelines will help your application scale on current and future devices: 
- ➤ Keep the number of threads per block a multiple of warp size (32). 
- ➤ Avoid small block sizes: Start with at least 128 or 256 threads per block. 
- ➤ Adjust block size up or down according to kernel resource requirements.
- ➤ Keep the number of blocks much greater than the number of SMs to expose suffi cient parallelism to your device.
- ➤ Conduct experiments to discover the best execution confi guration and resource usage.

### Synchronization

In CUDA, barrier synchronization can be performed at two levels:

- **System-level:** Wait for all work on both the host and the device to complete
- **Block-level:** Wait for all threads in a thread block to reach the same point in execution